In [1]:
import os
os.chdir("../")

In [21]:
import torch
import tslearn
import numpy as np
import torch

In [8]:
from pyts.image import GramianAngularField, MarkovTransitionField

In [65]:
from pyts.datasets.load import load_basic_motions
from typing import Tuple

In [5]:
data = load_basic_motions()

In [7]:
data.keys()

dict_keys(['data_train', 'target_train', 'data_test', 'target_test', 'DESCR', 'url'])

In [9]:
X = data["data_train"]

In [10]:
X.shape

(40, 6, 100)

In [11]:
tr = GramianAngularField(1.0)

In [14]:
X_tr = tr.transform(X.reshape(40*6,-1))

In [15]:
import matplotlib.pyplot as plt

In [ ]:
X_mtm = _markov_transition_matrix(X_binned, n_samples,
                                    n_timestamps, self.n_bins)
sum_mtm = X_mtm.sum(axis=2)
np.place(sum_mtm, sum_mtm == 0, 1)
X_mtm /= sum_mtm[:, :, None]

X_mtf = _markov_transition_field(
    X_binned, X_mtm, n_samples, n_timestamps, self.n_bins
)

window_size, remainder = divmod(n_timestamps, image_size)
if remainder == 0:
    X_amtf = np.reshape(
        X_mtf, (n_samples, image_size, window_size,
                image_size, window_size)
    ).mean(axis=(2, 4))
else:
    window_size += 1
    start, end, _ = segmentation(
        n_timestamps, window_size, self.overlapping, image_size
    )
    X_amtf = _aggregated_markov_transition_field(
        X_mtf, n_samples, image_size, start, end
    )
if self.flatten:
    return X_amtf.reshape(n_samples, -1)
return X_amtf

In [69]:
@torch.jit.script
def minmax_scaler(X: torch.Tensor, range: Tuple[float, float] = (-1, 1)) -> torch.Tensor:
    '''
        Scales the last dimension of X into the given range X has shape (...,d)
        output has the same shape but the last dimension is scaled to the range
    '''
    X_min = X.min(dim=-1, keepdim=True).values
    X_std = (X - X_min)/(X.max(dim=-1, keepdim=True).values - X_min)
    return X_std * (range[1] - range[0]) + range[0]

In [ ]:
def _markov_transition_matrix(X_binned, n_samples, n_timestamps, n_bins):
    X_mtm = np.zeros((n_samples, n_bins, n_bins))
    for i in range(n_samples):
        for j in range(n_timestamps - 1):
            X_mtm[i, X_binned[i, j], X_binned[i, j + 1]] += 1
    return X_mtm


@njit()
def _markov_transition_field(
    X_binned, X_mtm, n_samples, n_timestamps, n_bins
):
    X_mtf = np.zeros((n_samples, n_timestamps, n_timestamps))
    for i in prange(n_samples):
        for j in prange(n_timestamps):
            for k in prange(n_timestamps):
                X_mtf[i, j, k] = X_mtm[i, X_binned[i, j], X_binned[i, k]]
    return X_mtf


@njit()
def _aggregated_markov_transition_field(X_mtf, n_samples, image_size,
                                        start, end):
    X_amtf = np.empty((n_samples, image_size, image_size))
    for i in prange(n_samples):
        for j in prange(image_size):
            for k in prange(image_size):
                X_amtf[i, j, k] = np.mean(
                    X_mtf[i, start[j]:end[j], start[k]:end[k]]
                )
    return X_amtf

In [ ]:
def mtf_compute(X: torch.Tensor, bins: int = 50, scaling: Tuple[float, float] = (-1, 1)) -> torch.Tensor:
    '''
        Computes the Markov Transition Field of time series X, per sample, per channel
        input has shape (...,N, d, window_size) (any number of leading dimensions)
        output has shape (..., N, d, window_size, window_size)

        Bins are uniformly spaced then computes the MTF
    '''

    X_scaled = minmax_scaler(X, scaling)
    bins_boundaries = torch.arange(-1, 1, 2/(bins - 1))

    X_binned = torch.bucketize(X_scaled, bins_boundaries)
    n_bins = X_binned.shape[-1]

    # compute markov transition matrix
    current_bin = X_binned[..., :-1].view(-1, n_bins)
    next_bin = X_binned[..., 1:].view(-1, n_bins)

    X_mtm = torch.zeros(X_binned.shape[:-1] + (n_bins, n_bins))
    original_shape = X_mtm.shape
    X_mtm = X_mtm.reshape(-1, X_mtm.shape[-1], X_mtm.shape[-1]) # (n, bins, bins)

    X_mtm.view(-1, X_mtm.shape[-1], X_mtm.shape[-1]).scatter_add(1, current_bin, torch.ones_like(current_bin))
    X_mtm.view(-1, X_mtm.shape[-1], X_mtm.shape[-1]).scatter_add(2, next_bin, torch.ones_like(next_bin))

    for i in range(X_mtm.shape[0]):
        X_mtm[i, current_bin[i], next_bin[i]] += 1

    X_mtm = X_mtm.reshape(original_shape)

In [80]:
a = torch.randn(10, 5, 25)

In [81]:
a_scaled = minmax_scaler(a, (-1, 1))

In [106]:
a_binned = torch.bucketize(a_scaled, torch.arange(-1, 1, 2/(30 - 1)))

In [108]:
a_binned.min()

tensor(0)

In [109]:
a_current = a_binned[..., :-1]
a_next = a_binned[..., 1:]

In [110]:
a.shape + a.shape[-1:]

torch.Size([10, 5, 25, 25])

In [111]:
def _markov_transition_matrix(X_binned, n_samples, n_timestamps, n_bins):
    X_mtm = np.zeros((n_samples, n_bins, n_bins))
    for i in range(n_samples):
        for j in range(n_timestamps - 1):
            X_mtm[i, X_binned[i, j], X_binned[i, j + 1]] += 1
    return X_mtm

In [179]:
a_mtm = torch.zeros(a_binned.shape + (30, 30))

In [188]:
a_mtm.shape

torch.Size([10, 5, 25, 30, 30])

In [145]:
a_mtm.dtype

torch.float32

In [184]:
indices = 30 * a_current.reshape(-1, 24) + a_next.reshape(-1, 24)

In [185]:
indices.shape

torch.Size([50, 24])

In [191]:
a_mtm.view(50, 25, a_mtm.shape[-1]*a_mtm.shape[-1]).shape

torch.Size([50, 25, 900])

In [181]:
a_mtm.view(-1, a_mtm.shape[-1]*a_mtm.shape[-1]).scatter_add_(1, indices, torch.ones_like(a_mtm))
print()

RuntimeError: Index tensor must have the same number of dimensions as src tensor

In [171]:
a_mtm[0, 0].sum()

tensor(240.)

In [157]:
a_binned.shape

torch.Size([10, 5, 25])

In [158]:
50*25

1250

In [159]:
a_mtm.sum()

tensor(2400.)

In [170]:
_markov_transition_matrix(a_binned.reshape(50, 25), 50, 25, 30)[0].sum()

24.0

In [77]:
torch.arange(-1, 1, 1/50).shape

torch.Size([100])